In [150]:
import pandas

# import geopy and specif which geolocator to use
from geopy.geocoders import *
gl = Nominatim()

In [151]:
df=pandas.read_json('supermarkets.json')
df

,Address,City,Country,Employees,ID,Name,State
0,3666 21st St,San Francisco,USA,8,1,Madeira,CA 94114
1,735 Dolores St,San Francisco,USA,15,2,Bready Shop,CA 94119
2,332 Hill St,San Francisco,USA,25,3,Super River,California 94114
3,3995 23rd St,San Francisco,USA,10,4,Ben's Shop,CA 94114
4,1056 Sanchez St,San Francisco,USA,12,5,Sanchez,California
5,551 Alvarado St,San Francisco,USA,20,6,Richvalley,CA 94114


In [152]:
# Combine all address fields to get a full adress for geolocator
df['Address'] = df['Address'] + ', ' + df['City'] + ', ' + df['State'] + ', ' + df['Country']
df

,Address,City,Country,Employees,ID,Name,State
0,"3666 21st St, San Francisco, CA 94114, USA",San Francisco,USA,8,1,Madeira,CA 94114
1,"735 Dolores St, San Francisco, CA 94119, USA",San Francisco,USA,15,2,Bready Shop,CA 94119
2,"332 Hill St, San Francisco, California 94114, USA",San Francisco,USA,25,3,Super River,California 94114
3,"3995 23rd St, San Francisco, CA 94114, USA",San Francisco,USA,10,4,Ben's Shop,CA 94114
4,"1056 Sanchez St, San Francisco, California, USA",San Francisco,USA,12,5,Sanchez,California
5,"551 Alvarado St, San Francisco, CA 94114, USA",San Francisco,USA,20,6,Richvalley,CA 94114


In [153]:
# we can use functions with dataframe's data by using ".apply"!!
df['Coordinates']=df['Address'].apply(gl.geocode)

# We can also call column without brackets :) 
df.Coordinates

# df['Longitude']=df['Coordinates'][0].latitude
# df['Longitude']

0    (3666, 21st Street, Noe Valley, SF, California...
1                                                 None
2    (332, Hill Street, Noe Valley, SF, California,...
3    (3995, 23rd Street, Noe Valley, SF, California...
4    (1056, Sanchez Street, Noe Valley, SF, Califor...
5    (551, Alvarado Street, Noe Valley, SF, Califor...
Name: Coordinates, dtype: object

In [154]:
# we can use functions with dataframe's data by using ".apply"!!
# First we need to create a column containing geolocated data
df['Coordinates']=df['Address'].apply(gl.geocode)

# We can then extract latitude and longitude data from this column ...
df['Latitude']=df.Coordinates.apply(lambda x: x.latitude if x != None else 'Unknown')
df['Longitude']=df.Coordinates.apply(lambda x: x.longitude if x != None else 'Unknown')

# And then drop it - and restore the address back to how it was!
df=df.drop('Coordinates', 1)
df['Address']=df.Address.apply(lambda x: x.split(',')[0])
df

,Address,City,Country,Employees,ID,Name,State,Latitude,Longitude
0,3666 21st St,San Francisco,USA,8,1,Madeira,CA 94114,37.7565,-122.429
1,735 Dolores St,San Francisco,USA,15,2,Bready Shop,CA 94119,Unknown,Unknown
2,332 Hill St,San Francisco,USA,25,3,Super River,California 94114,37.7557,-122.429
3,3995 23rd St,San Francisco,USA,10,4,Ben's Shop,CA 94114,37.753,-122.432
4,1056 Sanchez St,San Francisco,USA,12,5,Sanchez,California,37.7521,-122.43
5,551 Alvarado St,San Francisco,USA,20,6,Richvalley,CA 94114,37.7537,-122.433


In [155]:
# Now lets fix everything else using lambda!
df['ZIP']=df.State.apply(lambda x: x.split(' ')[1] if len(x.split(' ')) > 1 else 'N/A')
df['State']=df.State.apply(lambda x: x.split(' ')[0] if len(x.split(' ')) > 1 else x)
df

,Address,City,Country,Employees,ID,Name,State,Latitude,Longitude,ZIP
0,3666 21st St,San Francisco,USA,8,1,Madeira,CA,37.7565,-122.429,94114
1,735 Dolores St,San Francisco,USA,15,2,Bready Shop,CA,Unknown,Unknown,94119
2,332 Hill St,San Francisco,USA,25,3,Super River,California,37.7557,-122.429,94114
3,3995 23rd St,San Francisco,USA,10,4,Ben's Shop,CA,37.753,-122.432,94114
4,1056 Sanchez St,San Francisco,USA,12,5,Sanchez,California,37.7521,-122.43,N/A
5,551 Alvarado St,San Francisco,USA,20,6,Richvalley,CA,37.7537,-122.433,94114


In [160]:
# And now let's save the nice version, and a different separator!
df.to_csv('shops_coordinates.sv',sep=':')